# <font color = 'indianred'>**Decoder_Base_Language_Head_Inference_Binary** </font>




















# <font color = 'indianred'> **Setting up the Environment** </font>



In [1]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  !pip install numpy -U -qq
  !pip install transformers evaluate wandb datasets accelerate trl peft bitsandbytes vllm -U -qq
  !pip uninstall tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 126.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is incompatible.

 <Font size = 5 color = 'indianred'>**Restart the session before moving onto next cell**
> Runtime- Restart Session

<font color = 'indianred'> *Load Libraries* </font>

In [1]:
import os
os.environ['VLLM_USE_V1'] = '1' # this should be done befre any other import

import torch
import torch.nn as nn
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from vllm.sampling_params import GuidedDecodingParams

INFO 11-03 02:02:29 [__init__.py:216] Automatically detected platform cuda.


In [2]:
# standard pythgion libraries
from pathlib import Path

from typing import Dict, List, Union, Optional, Tuple
from tqdm import tqdm
import json
import os
import sys

import numpy as np
import seaborn as sns
import re
import gc
import time

# Data Science librraies
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score
from pprint import pprint

# Pytorch
import torch

# Huggingface Librraies
import evaluate
from datasets import load_dataset

from huggingface_hub import login

# Logging and secrets
import wandb
from google.colab import userdata
from huggingface_hub import login, HfApi, create_repo

In [3]:
# Determine the storage location based on the execution environment
# If running on Google Colab, use Google Drive as storage
# CHANGE FOLDERS TO WHERE YOU WANT TO SAVE DATA AND MODELS

if 'google.colab' in str(get_ipython()):
    from google.colab import drive  # Import Google Drive mounting utility
    drive.mount('/content/drive')  # Mount Google Drive

    # Set base folder path for storing data on Google Drive
    base_folder= Path('/content/drive/MyDrive/data')

# If running locally, specify a different path
else:
    # Set base folder path for storing data on local machine
    base_folder= Path('/Users/ronchaudhuri/Documents/Models/Data')

Mounted at /content/drive


In [4]:
wandb_api_key = userdata.get('WANDB_API_KEY')
hf_token = userdata.get('HF_Token')

In [5]:
if hf_token:
    # Log in to Hugging Face
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in notebook secrets.")

Successfully logged in to Hugging Face!


In [6]:
if wandb_api_key:
  wandb.login(key=wandb_api_key)
  print("Successfully logged in to WANDB!")
else:
    print("WANDB key not found in notebook secrets.")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ronchaudhuri29 (ronchaudhuri29-the-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to WANDB!


# <font color = 'indianred'> **Load Data set**
    


In [7]:
val_split = load_dataset('Ron00769/jigsaw_binary_base_language_head', split = 'valid')
val_prompts = val_split['prompt']
test_split = load_dataset('Ron00769/jigsaw_binary_base_language_head', split = 'test')
test_prompts = test_split['prompt']
print()
pprint(val_split[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/538 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/245k [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/32.6k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/33.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1623 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/203 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/203 [00:00<?, ? examples/s]


{'completion': ' violates',
 'prompt': 'Does the following Reddit comment violate the subreddit rules? '
           "Answer 'complies' or 'violates'.\n"
           '\n'
           'Comment: Comment: Hello \n'
           'my name is Ali Hadii am Yemeni but i am living in Saudi arabia '
           'with student visa i have finished my education and required to go '
           'back to yaman which i dont want to do fearing for my life due to '
           'Yemeni Civil War i need help seeking asylum in Canada if you can '
           'help in any way send me a message please i have no money and no '
           'one .\n'
           ' | Rule: No legal advice: Do not offer or request legal advice. | '
           'Subreddit: canada\n'
           'Answer:',
 'tag': 1}


#  <font color = 'indianred'> **Download Model and setup vLLM**

In [8]:
torch.cuda.empty_cache()
BASE_MODEL = "google/gemma-2-2b"
LORA_ADAPTER = "Ron00769/jigsaw_binary_base_language_head"

In [9]:
llm = LLM(
    model=BASE_MODEL,
    dtype=torch.bfloat16,
    trust_remote_code=True,
    enable_lora=True,
    max_lora_rank=128,
    gpu_memory_utilization=0.6,
    guided_decoding_backend="outlines",
)


INFO 11-03 02:04:16 [utils.py:233] non-default args: {'trust_remote_code': True, 'dtype': torch.bfloat16, 'gpu_memory_utilization': 0.6, 'disable_log_stats': True, 'enable_lora': True, 'max_lora_rank': 128, 'guided_decoding_backend': 'outlines', 'model': 'google/gemma-2-2b'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

INFO 11-03 02:04:33 [model.py:547] Resolved architecture: Gemma2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

WARNING 11-03 02:04:38 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-03 02:06:46 [llm.py:306] Supported_tasks: ['generate']


In [10]:
lora_request = LoRARequest(
    lora_name="jigsaw_binary_lora",
    lora_int_id=1,
    lora_path=LORA_ADAPTER
)

GUIDE_REGEX = "(complies|violates)"
guided_decoding = GuidedDecodingParams(regex=GUIDE_REGEX)


sampling_params = SamplingParams(
    temperature=0,
    max_tokens=2,
    guided_decoding=guided_decoding,
)

/tmp/ipython-input-1960497631.py:11: DeprecationWarning: guided_decoding is deprecated. This will be removed in v0.12.0 or v1.0.0, which ever is soonest. Please use structured_outputs instead.
  sampling_params = SamplingParams(


# <font color = 'indianred'> **Setup WandB**</font>

In [11]:
%env WANDB_PROJECT=jigsaw_language_head_2025

run = wandb.init(
    project="jigsaw_language_head_2025",
    entity="ronchaudhuri29-the-university-of-texas-at-dallas",
    id="gucl8k4n",
    resume="must"
)


env: WANDB_PROJECT=jigsaw_language_head_2025


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


#<font color = 'indianred'> **Utility Function**

In [12]:
def generate_predictions_batch(prompts: List[str],
                               llm: LLM,
                               sampling_params: SamplingParams,
                               lora_request: LoRARequest,
                               batch_size: int = 64) -> List[int]:
    """
    Generate predictions for a list of prompts using vLLM batch inference.

    Args:
        prompts: List of input prompts
        llm: vLLM model instance
        sampling_params: Sampling parameters for generation
        lora_request: LoRA adapter request
        batch_size: Batch size for inference

    Returns:
         List of predicted labels as strings ("complies" or "violates")
    """
    predictions = []

    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating predictions"):
        batch = prompts[i:i+batch_size]

        outputs = llm.generate(
            batch,
            sampling_params,
            lora_request=lora_request
        )

        for o in outputs:
            text = (o.outputs[0].text or "").strip().lower()
            if "violates" in text:
                predictions.append("violates")
            elif "complies" in text:
                predictions.append("complies")
            else:
                predictions.append("unknown")

    print(f"Processed {len(predictions)}/{len(prompts)} prompts")
    return predictions


In [13]:
f1 = evaluate.load("f1")
def compute_metrics(labels, preds):
    f1_macro = f1.compute(predictions=preds, references=labels, average="macro")
    return {"f1_macro": f1_macro["f1"]}

In [14]:
def evaluate_binary_predictions(true_labels: List[str],
                                predictions: List[str],
                                class_names: List[str],
                                split_name: str = "valid",
                                log_to_wandb: bool = True,
                                show_confusion_matrix: bool = True) -> Dict:
    """
    Evaluate binary predictions ('complies' vs 'violates') and optionally log to wandb.

    Args:
        true_labels: List of true labels ("complies" or "violates")
        predictions: List of predicted labels ("complies" or "violates")
        class_names: List of all possible class names
        split_name: Name of the split (for wandb logging)
        log_to_wandb: Whether to log metrics to wandb
        show_confusion_matrix: Whether to display confusion matrix

    Returns:
        Dictionary containing evaluation metrics
    """
    # Convert labels to lowercase
    true_labels = [str(l).strip().lower() for l in true_labels]
    predictions = [str(p).strip().lower() for p in predictions]

    # Print samples and shapes
    print(f"\n{'='*60}")
    print(f"{split_name.upper()} SET EVALUATION")
    print(f"{'='*60}")
    print(f"Sample true labels: {true_labels[:3]}")
    print(f"Sample predictions: {predictions[:3]}")

    # Compute metrics
    f1_macro = f1_score(true_labels, predictions, average="macro")
    acc = accuracy_score(true_labels, predictions)

    print(f"\nMetrics:")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  F1 Macro: {f1_macro:.4f}")

    # Log to wandb
    if log_to_wandb:
        wandb.log({
            f"{split_name}_accuracy": acc,
            f"{split_name}_f1_macro": f1_macro
        })
        print(f"  ✓ Logged to wandb")

    # Display confusion matrix
    if show_confusion_matrix:
        print(f"\nConfusion Matrix:")
        cm = confusion_matrix(true_labels, predictions, labels=class_names)
        print(cm)

    return {"accuracy": acc, "f1_macro": f1_macro}

#<font color = 'indianred'> **Performance on Validation Set**

In [15]:
# Define class names (used for both validation and test)
class_names = ["complies", "violates"]

In [16]:
val_predictions = generate_predictions_batch(
    prompts=val_prompts, # Extract the prompt string
    llm=llm,
    sampling_params=sampling_params,
    lora_request=lora_request,
    batch_size=64
)

Generating predictions:   0%|          | 0/4 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/64 [00:00<?, ?it/s]

WARNING 11-03 02:07:29 [processor.py:215] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.


Processed prompts:   0%|          | 0/64 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generating predictions:  25%|██▌       | 1/4 [00:32<01:36, 32.24s/it]

...

Generating predictions:  75%|███████▌  | 3/4 [00:32<00:07,  7.41s/it]

Adding requests:   0%|          | 0/11 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/11 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generating predictions: 100%|██████████| 4/4 [00:32<00:00,  8.21s/it]

Processed 203/203 prompts


In [17]:
val_predictions[0:5]

['violates', 'complies', 'complies', 'violates', 'complies']

In [18]:
# Evaluate
valid_metrics = evaluate_binary_predictions(
    true_labels=val_split['completion'],
    predictions=val_predictions,
    class_names=class_names,
    split_name="valid",
    log_to_wandb=True,
    show_confusion_matrix=True
)


VALID SET EVALUATION
Sample true labels: ['violates', 'violates', 'complies']
Sample predictions: ['violates', 'complies', 'complies']

Metrics:
  Accuracy: 0.6453
  F1 Macro: 0.4177
  ✓ Logged to wandb

Confusion Matrix:
[[92 10]
 [57 39]]


#  <font color = 'indianred'> **Performance on Test Set** </font>

In [19]:
# Generate predictions
test_predictions = generate_predictions_batch(
    prompts=test_prompts,
    llm=llm,
    sampling_params=sampling_params,
    lora_request=lora_request,
    batch_size=128  # Can use larger batch for test
)

Generating predictions:   0%|          | 0/2 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/128 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/128 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generating predictions:  50%|█████     | 1/2 [00:00<00:00,  2.06it/s]

Adding requests:   0%|          | 0/75 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/75 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generating predictions: 100%|██████████| 2/2 [00:00<00:00,  2.56it/s]

Processed 203/203 prompts


In [20]:
# Evaluate
test_metrics = evaluate_binary_predictions(
    true_labels=test_split['completion'],
    predictions=test_predictions,
    class_names=class_names,
    split_name="test",
    log_to_wandb=True,
    show_confusion_matrix=True
)


TEST SET EVALUATION
Sample true labels: ['complies', 'violates', 'complies']
Sample predictions: ['complies', 'complies', 'complies']

Metrics:
  Accuracy: 0.5961
  F1 Macro: 0.3879
  ✓ Logged to wandb

Confusion Matrix:
[[81 15]
 [65 40]]


In [21]:
# Finish wandb run
wandb.finish()

test_accuracy,▁
test_f1_macro,▁
valid_accuracy,▁
valid_f1_macro,▁
eval/entropy,2.49263
eval/loss,0.3018
eval/mean_token_accuracy,0.85294
eval/num_tokens,327473
eval/runtime,20.3923
eval/samples_per_second,9.955
eval/steps_per_second,5.002


In [22]:
text = val_split['prompt'][0]
text

"Does the following Reddit comment violate the subreddit rules? Answer 'complies' or 'violates'.\n\nComment: Comment: Hello \nmy name is Ali Hadii am Yemeni but i am living in Saudi arabia with student visa i have finished my education and required to go back to yaman which i dont want to do fearing for my life due to Yemeni Civil War i need help seeking asylum in Canada if you can help in any way send me a message please i have no money and no one .\n | Rule: No legal advice: Do not offer or request legal advice. | Subreddit: canada\nAnswer:"

# <font color = 'indianred'> **Inference** </font>

# <font color = 'indianred'> **Load Data set**

In [23]:
data_folder = base_folder/'datasets'
df_inference = pd.read_csv(data_folder/'test.csv')
df_inference

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...
5,2034,Top 10 Pentakills in August https://youtu.be/Z...,"No Advertising: Spam, referral links, unsolici...",leagueoflegends,code free tyrande --->>> [Imgur](http://i.imgu...,see tits sweet at this point http://sh.or...,its my asreddit https://www.youtube.com/watch?...,click here for more videos[Russian mafia messe...
6,2035,WIN A SAMSUNG SMARTWATCH!\n\nhttp://www.innomd...,"No Advertising: Spam, referral links, unsolici...",AskReddit,"[ENJOY 50,000 BONUS POINTS after $4,000 spent ...",We have high quality Canadian Polymer banknote...,i was on the same ios and try this jailbreak m...,We're streaming Pokemon Veitnamese Crystal RIG...
7,2036,[This mixtape is lit FAM!](https://beholdthear...,"No Advertising: Spam, referral links, unsolici...",BlackPeopleTwitter,watch good one hooters there http://https://x5...,bad pussies waiting for you herein http://gour...,[Full HD Movie Online Free](http://www.flickma...,* Computers :http://livematchstreamz.blogspot....
8,2037,must be watch movie https://sites.google.com/s...,"No Advertising: Spam, referral links, unsolici...",movies,see there hooters sweet http://url.t.aylo...,"Sign up with my link and get $1,000 of free pr...",The fact that there are pro and cons to everyt...,Oh that's interesting. You should check out my...
9,2038,Free paypal cards here!! https://www.pointspri...,"No Advertising: Spam, referral links, unsolici...",pics,she will come your home open her legs with an...,Old and young your girl-next-door make sex wi...,"i think the conjuring 2 is not bad movie ,The...","wow, she's so beautyful\n>[Sakura Vietnam](htt..."


In [24]:
# Identify columns
ID_COL = 'row_id'
TEXT_COL = 'body'

#  <font color = 'indianred'> **Build Prompt**

In [25]:
def build_prompt(text):
    return (
        "Does the following Reddit comment violate the subreddit rules? "
        "Answer 'complies' or 'violates'.\n\n"
        f"Comment: {str(text).strip()}\n"
        "Answer:"
    )

In [26]:
prompts = [build_prompt(t) for t in df_inference[TEXT_COL]]

In [27]:
preds = generate_predictions_batch(
    prompts=prompts,                #full list built from df_inference['body']
    llm=llm,
    sampling_params=sampling_params,
    lora_request=lora_request,
    batch_size=128
)

Generating predictions:   0%|          | 0/1 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generating predictions: 100%|██████████| 1/1 [00:00<00:00, 12.94it/s]

Processed 10/10 prompts


In [28]:
print(preds[:10])

['complies', 'complies', 'violates', 'complies', 'complies', 'complies', 'complies', 'complies', 'complies', 'complies']
